In [2]:
# Import necessary libraries

import numpy as np
import pandas as pd # Pandas for data manipulation in DataFrame format
import matplotlib.pyplot as plt # for plotting
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from scipy.interpolate import UnivariateSpline


# SECTION: Functions for computation


In [3]:
def compute_rotations(theta_x, theta_z):
    """
    Compute rotation matrices for a body attached to a U-Joint
    based on the angles of rotation around the x and z axes.
    Parameters
    ----------
    theta_x : array-like
        Angles of rotation around the x-axis in radians (global reference frame).
    theta_z : array-like
        Angles of rotation around the z-axis in radians (global reference frame).
    Returns
    -------
    rotation_matrices : np.ndarray
        Array of rotation matrices corresponding to the input angles.
    """
    
    # Initialize a list to store rotation matrices
    rotation_matrices = []
    
    # Iterate over the angles    
    for x, z in zip(theta_x, theta_z):
        
        # Rotation around the z-axis
        Rz = np.array([
            [np.cos(z), -np.sin(z),  0],
            [np.sin(z),  np.cos(z),  0],
            [        0,          0,  1]
            ])
        
        # Rotation around the x-axis
        Rx = np.array([
            [1,         0,          0],
            [0, np.cos(x), -np.sin(x)],
            [0, np.sin(x),  np.cos(x)]
            ])
        
        # Multimply both matrices to get the final rotation matrix 
        R_matrix = Rx @ Rz

        # Append the rotation matrix to the list
        rotation_matrices.append(R_matrix)

    # Convert the list of rotation matrices to a numpy array    
    rotation_matrices = np.array(rotation_matrices)
    
    return rotation_matrices


In [4]:
def compute_positions(rotation_matrices, rod_vector, P_0):
    """
    Compute the positions of the end of the rod based on 
    the rotation matrices, direction vector and origin position.
    Parameters
    ----------
    rotation_matrices : np.ndarray
        Array of rotation matrices corresponding to the input angles.
    rod_vector : array-like
        Direction vector of the rod in the body frame.
    P_0 : array-like
        Origin position of the rod in the global reference frame.
    Returns
    -------
    rod_positions : np.ndarray
        Array of positions of the end of the rod in the global reference frame.
    """
    
    # Initialize a list to store the positions of the rods
    rod_positions = []
    
    for R in rotation_matrices:
        # Compute the position of the rod
        position = P_0 + R @ rod_vector
        
        # Append the position to the list
        rod_positions.append(position)
    
    # Convert the list of rod positions to a numpy array    
    rod_positions = np.array(rod_positions)
    
    return rod_positions


In [5]:
def compute_derivatives(df):
    """
    Compute the first and second derivatives of the angles of rotation
    around the x and z axes for each U-Joint in the dataframe.
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the time and angles of rotation for each U-Joint.
    Returns
    -------
    df : pd.DataFrame
        DataFrame with additional columns for the first and second derivatives of the angles.
    """

    # Extract the angles and time from the dataframe
    time = df['Time'].to_numpy()
    U1_theta_x = df['U1Rx'].to_numpy()
    U1_theta_z = df['U1Rz'].to_numpy()
    U2_theta_x = df['U2Rx'].to_numpy()
    U2_theta_z = df['U2Rz'].to_numpy()
    U3_theta_x = df['U3Rx'].to_numpy()
    U3_theta_z = df['U3Rz'].to_numpy()

    array_length = len(U1_theta_x)

    U1_dtheta_x = np.zeros(array_length)
    U2_dtheta_x = np.zeros(array_length)
    U3_dtheta_x = np.zeros(array_length)
    U1_dtheta_z = np.zeros(array_length)
    U2_dtheta_z = np.zeros(array_length)
    U3_dtheta_z = np.zeros(array_length)

    U1_ddtheta_x = np.zeros(array_length)
    U2_ddtheta_x = np.zeros(array_length)
    U3_ddtheta_x = np.zeros(array_length)
    U1_ddtheta_z = np.zeros(array_length)
    U2_ddtheta_z = np.zeros(array_length)
    U3_ddtheta_z = np.zeros(array_length)

    # Compute the first derivatives
    # Central differences for the middle points    
    for i in range(1, len(U1_theta_x)-1):
        dt = time[i+1] - time[i-1]
        U1_dtheta_x[i] = (U1_theta_x[i+1] - U1_theta_x[i-1]) / dt
        U1_dtheta_z[i] = (U1_theta_z[i+1] - U1_theta_z[i-1]) / dt
        U2_dtheta_x[i] = (U2_theta_x[i+1] - U2_theta_x[i-1]) / dt
        U2_dtheta_z[i] = (U2_theta_z[i+1] - U2_theta_z[i-1]) / dt
        U3_dtheta_x[i] = (U3_theta_x[i+1] - U3_theta_x[i-1]) / dt
        U3_dtheta_z[i] = (U3_theta_z[i+1] - U3_theta_z[i-1]) / dt
    
    # Forward differences for the first frame
    dt_0 = time[1] - time[0]
    U1_dtheta_x[0] = (U1_theta_x[1] - U1_theta_x[0]) / dt_0
    U1_dtheta_z[0] = (U1_theta_z[1] - U1_theta_z[0]) / dt_0
    U2_dtheta_x[0] = (U2_theta_x[1] - U2_theta_x[0]) / dt_0
    U2_dtheta_z[0] = (U2_theta_z[1] - U2_theta_z[0]) / dt_0
    U3_dtheta_x[0] = (U3_theta_x[1] - U3_theta_x[0]) / dt_0
    U3_dtheta_z[0] = (U3_theta_z[1] - U3_theta_z[0]) / dt_0

    # Backwar differences for the last frame
    dt_f = time[-1] - time[-2]
    U1_dtheta_x[-1] = (U1_theta_x[-1] - U1_theta_x[-2]) / dt_f
    U1_dtheta_z[-1] = (U1_theta_z[-1] - U1_theta_z[-2]) / dt_f
    U2_dtheta_x[-1] = (U2_theta_x[-1] - U2_theta_x[-2]) / dt_f
    U2_dtheta_z[-1] = (U2_theta_z[-1] - U2_theta_z[-2]) / dt_f
    U3_dtheta_x[-1] = (U3_theta_x[-1] - U3_theta_x[-2]) / dt_f
    U3_dtheta_z[-1] = (U3_theta_z[-1] - U3_theta_z[-2]) / dt_f

    # Compute the second derivatives
    # Central differences for the middle points 
    for i in range(1, len(U1_dtheta_x)-1):
        dt = time[i+1] - time[i-1]
        U1_ddtheta_x[i] = (U1_dtheta_x[i+1] - U1_dtheta_x[i-1]) / dt
        U2_ddtheta_x[i] = (U2_dtheta_x[i+1] - U2_dtheta_x[i-1]) / dt
        U3_ddtheta_x[i] = (U3_dtheta_x[i+1] - U3_dtheta_x[i-1]) / dt
        U1_ddtheta_z[i] = (U1_dtheta_z[i+1] - U1_dtheta_z[i-1]) / dt
        U2_ddtheta_z[i] = (U2_dtheta_z[i+1] - U2_dtheta_z[i-1]) / dt
        U3_ddtheta_z[i] = (U3_dtheta_z[i+1] - U3_dtheta_z[i-1]) / dt

    # Forward differences for the first frame
    dt_0 = time[1] - time[0]
    U1_ddtheta_x[0] = (U1_dtheta_x[1] - U1_dtheta_x[0]) / dt_0
    U1_ddtheta_z[0] = (U1_dtheta_z[1] - U1_dtheta_z[0]) / dt_0
    U2_ddtheta_x[0] = (U2_dtheta_x[1] - U2_dtheta_x[0]) / dt_0
    U2_ddtheta_z[0] = (U2_dtheta_z[1] - U2_dtheta_z[0]) / dt_0
    U3_ddtheta_x[0] = (U3_dtheta_x[1] - U3_dtheta_x[0]) / dt_0
    U3_ddtheta_z[0] = (U3_dtheta_z[1] - U3_dtheta_z[0]) / dt_0

    # Backwar differences for the last frame
    dt_f = time[-1] - time[-2]
    U1_ddtheta_x[-1] = (U1_dtheta_x[-1] - U1_dtheta_x[-2]) / dt_f
    U1_ddtheta_z[-1] = (U1_dtheta_z[-1] - U1_dtheta_z[-2]) / dt_f
    U2_ddtheta_x[-1] = (U2_dtheta_x[-1] - U2_dtheta_x[-2]) / dt_f
    U2_ddtheta_z[-1] = (U2_dtheta_z[-1] - U2_dtheta_z[-2]) / dt_f
    U3_ddtheta_x[-1] = (U3_dtheta_x[-1] - U3_dtheta_x[-2]) / dt_f
    U3_ddtheta_z[-1] = (U3_dtheta_z[-1] - U3_dtheta_z[-2]) / dt_f

    # add the derivatives to the dataframe
    df['U1_dtheta_x'] = U1_dtheta_x
    df['U1_dtheta_z'] = U1_dtheta_z
    df['U2_dtheta_x'] = U2_dtheta_x
    df['U2_dtheta_z'] = U2_dtheta_z
    df['U3_dtheta_x'] = U3_dtheta_x
    df['U3_dtheta_z'] = U3_dtheta_z
    df['U1_ddtheta_x'] = U1_ddtheta_x
    df['U1_ddtheta_z'] = U1_ddtheta_z
    df['U2_ddtheta_x'] = U2_ddtheta_x
    df['U2_ddtheta_z'] = U2_ddtheta_z
    df['U3_ddtheta_x'] = U3_ddtheta_x
    df['U3_ddtheta_z'] = U3_ddtheta_z
    
    return df



In [6]:
def compute_joint_velocity(theta_x, theta_z, dtheta_x, dtheta_z, rod_vector):
    """
    Compute the velocity of a point on a body attached to a U-Joint
    based on the angles of rotation around the x and z axes.
    Parameters
    :theta_x: array-like
        Angles of rotation around the x-axis in radians in global reference frame.
    :theta_z: array-like
        Angles of rotation around the z-axis in radians in global reference frame.
    :dtheta_x: array-like
        Angular velocities around the x-axis in radians per second in global reference frame.
    :dtheta_z: array-like
        Angular velocities around the z-axis in radians per second in global reference frame.
    :rod_vector: array-like
        Vector the connects the end point and origin of the rod.
    :return: np.ndarray
        Array of velocities corresponding to the input angles and position vector.
    """
    # Initialize a list to store velocities
    velocities = []

    # Loop through the angles and their derivatives
    for x, z, dx, dz in zip(theta_x, theta_z, dtheta_x, dtheta_z):
        
        # Compute the Rotation matrices
        Rz = np.array([
            [np.cos(z), -np.sin(z), 0],
            [np.sin(z),  np.cos(z), 0],
            [0, 0, 1]
            ])
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(x), -np.sin(x)],
            [0, np.sin(x),  np.cos(x)]
            ])
        R = Rx @ Rz

        # Compute the angular velocity
        omega = Rx @ np.array([0, 0, dz]) + np.array([dx, 0, 0])

        # Compute the global position vector
        r = R @ rod_vector

        # Compute the velocity using the cross product
        velocity = np.cross(omega, r)
        
        # Append the velocity to the list
        velocities.append(velocity)

    return np.array(velocities)



In [7]:
def compute_joint_acceleration(theta_x, theta_z, dtheta_x, dtheta_z, ddtheta_x, ddtheta_z, rod_vector):
    """
    Compute the acceleration of a point on a body attached to a U-Joint
    based on the angles of rotation around the x and z axes.
    :theta_x: array-like
        Angles of rotation around the x-axis in radians.
    :theta_z: array-like
        Angles of rotation around the z-axis in radians.
    :dtheta_x: array-like
        Angular velocities around the x-axis in radians per second.
    :dtheta_z: array-like
        Angular velocities around the z-axis in radians per second.
    :ddtheta_x: array-like
        Angular accelerations around the x-axis in radians per second squared.
    :ddtheta_z: array-like
        Angular accelerations around the z-axis in radians per second squared.
    :rod_vector: array-like
        Vector the connects the end point and origin of the rod.
    """
    accelerations = []

    # Loop through the angles and their first and second derivatives
    for x, z, dx, dz, ddx, ddz in zip(theta_x, theta_z, dtheta_x, dtheta_z, ddtheta_x, ddtheta_z):
        # Rotation matrices
        Rz = np.array([
            [np.cos(z), -np.sin(z), 0],
            [np.sin(z),  np.cos(z), 0],
            [        0,          0, 1]
        ])
        Rx = np.array([
            [1,         0,          0],
            [0, np.cos(x), -np.sin(x)],
            [0, np.sin(x),  np.cos(x)]
        ])
        R = Rx @ Rz

        dRx = dx * np.array([
            [0,          0,          0],
            [0, -np.sin(x), -np.cos(x)],
            [0,  np.cos(x), -np.sin(x)]
        ])
        
        # Angular velocity
        omega = Rx @ np.array([0, 0, dz]) + np.array([dx, 0, 0])
        
        # Angular acceleration
        alpha = Rx @ np.array([0, 0, ddz]) + np.array([ddx, 0, 0]) + dRx @ np.array([0, 0, dz])

        # Compute the global position vector
        r = R @ rod_vector

        # Compute the acceleration using the cross product and the angular acceleration
        a = np.cross(alpha, r) + np.cross(omega, np.cross(omega, r))
        
        # Append the acceleration to the list
        accelerations.append(a)

    return np.array(accelerations)



In [8]:
def compute_orientation(P1, P2, P3):
    """
    Compute the Rotation matrix for the orientation defined 
    by the coordinates of 3 points.
    Parameters
    ----------
    P1 : array-like
        Coordinates of the first point in the global reference frame.
    P2 : array-like
        Coordinates of the second point in the global reference frame.
    P3 : array-like
        Coordinates of the third point in the global reference frame.
    Returns
    -------
    R_matrix : np.ndarray
        Rotation matrix corresponding to the orientation defined by the points.
    """
    # Define a lambda function to normalize vectors
    normalize = lambda x: x / np.linalg.norm(x)

    # Compute the normal vector of the plane defined by P1, P2, and P3
    v1 = normalize(P2 - P1)
    v2 = normalize(P3 - P1)
    
    # Define the first vector as the vector of P1 to P2
    # this will be the x-axis of the local frame
    X = v1
    
    # Compute the cross product to get the normal vector and normalize it
    # this will be the y-axis of the local frame
    Y = normalize(np.cross(v2, v1))
    
    # Compute the cross product to get the third vector and normalize it
    # this will be the z-axis of the local frame
    Z = normalize(np.cross(X, Y))

    # Build the rotation matrix
    R_matrix = np.array([X, Y, Z]).T
    
    return R_matrix



In [9]:
def compute_body_orientation(P1_list, P2_list, P3_list):
    """
    Compute the body orientation based on the positions of the spherical joints.
    Parameters
    ----------
    P1_list : list of array-like
        List of coordinates of the first point in the global reference frame.
    P2_list : list of array-like
        List of coordinates of the second point in the global reference frame.
    P3_list : list of array-like
        List of coordinates of the third point in the global reference frame.
    Returns
    -------
    rotation_matrices : np.ndarray
        Array of rotation matrices corresponding to the orientation defined by the points.  
    """
    # Initialize a list to store the rotation matrices
    rotation_matrices = []
    
    # Loop through the lists of points
    for P1, P2, P3 in zip(P1_list, P2_list, P3_list):
        # Compute the rotation matrix for each set of positions
        R_matrix = compute_orientation(P1, P2, P3)
        
        # Append the rotation matrix to the list
        rotation_matrices.append(R_matrix)
    
    # Convert the lists to a numpy array    
    rotation_matrices = np.array(rotation_matrices)
    
    return rotation_matrices



In [10]:
def compute_body_omega(rotation_matrices, df):
    """
    Compute angular velocity from a sequence of rotation matrices and time data.
    Parameters:
    ----------
    - rotation_matrices: Nx3x3 array of rotation matrices
    - df: pandas DataFrame with a 'Time' column
    Returns:
    -------
    - omega: Nx3 array of angular velocity vectors in global frame
    """
    # Extract time from the DataFrame
    time = df['Time'].to_numpy()
    N = len(time)
    
    # Initialize arrays for Rotation Matrices derivatives and angular velocities
    R_dots = np.zeros_like(rotation_matrices)
    omega = np.zeros((N, 3))

    # Compute the derivatives
    # Central differences for the middle points
    for i in range(1, N - 1):
        dt = time[i + 1] - time[i - 1]
        R_dots[i] = (rotation_matrices[i + 1] - rotation_matrices[i - 1]) / dt

    # Endpoints: forward/backward
    R_dots[0] = (rotation_matrices[1] - rotation_matrices[0]) / (time[1] - time[0])
    R_dots[-1] = (rotation_matrices[-1] - rotation_matrices[-2]) / (time[-1] - time[-2])

    # Compute omega from skew matrix
    for i in range(N):
        skew_omega = R_dots[i] @ rotation_matrices[i].T
        
        # The omega vector is derived from the skew-symmetric matrix
        # the elements either side of the diagonal are averaged to get the angular velocity vector
        omega[i] = 0.5 * np.array([
            skew_omega[2, 1] - skew_omega[1, 2],
            skew_omega[0, 2] - skew_omega[2, 0],
            skew_omega[1, 0] - skew_omega[0, 1]
        ])

    return omega



In [11]:
def compute_body_alpha(angular_velocities, df):
    """
    Compute angular acceleration from a sequence of angular velocities and time data.
    ----------
    Parameters:
    - angular_velocities: Nx3 array of angular velocity vectors
    - df: pandas DataFrame with a 'Time' column
    ----------
    Returns:
    - alpha: Nx3 array of angular acceleration vectors in global frame
    """
    # Extract time from the DataFrame
    time = df['Time'].to_numpy()
    N = len(time) # number of time steps
    
    # Initialize array for angular accelerations
    alpha = np.zeros((N, 3))

    # Compute the derivatives
    # Central differences for the middle points
    for i in range(1, N - 1):
        dt = time[i + 1] - time[i - 1]
        alpha[i] = (angular_velocities[i + 1] - angular_velocities[i - 1]) / dt

    # Endpoints: forward/backward
    alpha[0] = (angular_velocities[1] - angular_velocities[0]) / (time[1] - time[0])
    alpha[-1] = (angular_velocities[-1] - angular_velocities[-2]) / (time[-1] - time[-2])

    return alpha



In [12]:
def transform_global_to_local(vectors, rotation_matrices):
    """
    Transform global vectors into the local frame using rotation matrices.
    Parameters
    ---------- 
    vectors: Nx3 array of global vectors (e.g., angular velocity)
    rotation_matrices: Nx3x3 array of rotation matrices
    Returns
    ----------
    Nx3 array of vectors in local frames
    """
    # Performs a batch matrix-vector multiplication
    # between a sequence of rotation matrices and a sequence of vectors.
    return np.einsum('nij,nj->ni', rotation_matrices.transpose(0, 2, 1), vectors)



In [13]:
def tilde(v):
    """
    Compute the skew symmetric matrix of a 3x1 vector v.
    The skew symmetric matrix is defined as:
    [  0   -v[2]  v[1] ]
    [ v[2]   0   -v[0] ]
    [-v[1]  v[0]   0   ]
    """
    if len(v) != 3:
        raise ValueError("Input vector must be of length 3.")
    # Check if v is a numpy array, if not convert it to one
    # This is important for the case when v is a list or tuple
    if not isinstance(v, np.ndarray):
        v = np.array(v)
    
    # Compute the skew symmetric matrix
    v_tilde = np.array([
        [  0,  -v[2], v[1]],
        [ v[2],  0,  -v[0]],
        [-v[1], v[0],  0  ]
        ])
    
    return v_tilde



In [14]:
def tilde_list(vectors):
    """Compute the skew symmetric matrix representation of a list of 3x1 vectors."""
    
    # Initialize an empty list to store the skew symmetric matrices
    tildelist = []
    
    # Iterate over the list of vectors and compute the skew symmetric matrix for each
    for v in vectors:
        tildelist.append(tilde(v))
    
    return np.array(tildelist)



In [15]:
def build_matrix_A(omega, alpha):
    """
    Build the list of "matrices a" from the angular velocities and accelerations.
    These matrices are used to compute the forces acting on the body.
    Parameters
    ----------
    omega: array of angular velocities
    alpha: array of angular accelerations
    Returns
    ----------
    matrix_A: list of 3x3 "matrices a"
    """
    # Compute the skew symmetric matrices for omega and alpha    
    omega_tilde = tilde_list(omega)
    alpha_tilde = tilde_list(alpha)

    # Initialize the list to store the matrices 
    matrix_A = []

    # Iterate over the skew symmetric matrices and compute the "matrices a"
    for ot, at in zip(omega_tilde, alpha_tilde):
        # Skip rows where either ot or at is NaN
        if isinstance(ot, float) and np.isnan(ot) or isinstance(at, float) and np.isnan(at):
            matrix_A.append(np.nan)  # Append NaN for missing entries
        else:
            # the "matrix a" is computed as:
            # A = omega (cross) omega + skew(alpha) or:
            # A = skew(omega) @ skew(omega) + skew(alpha)
            mat = ot @ ot + at
            # append the computed matrix to the list
            matrix_A.append(mat)
    
    return matrix_A  # Return the list of matrices



In [16]:
def compute_r_PG(df, acl, g, mass):
    """
    Compute the vector from the reference point P to the center of gravity G (r_PG).

    Parameters:
    ----------
    df: DataFrame with joint forces in the local reference frame and A matrix data
    acl: vector of the accelereations of the reference point (Joint 1) in the local reference frame (3,)
    g: gravity vector in the local reference frame (3,)
    mass: scalar mass of the system in kg

    Returns:
    ----------
    r_PG: vector from the reference point P to G (ceter of gravity) in the local reference frame (3,)
    """
    # Extract joint forces into 3D vectors
    j1 = np.array([[j1x, j1y, j1z] for j1x, j1y, j1z in zip(df['J1_fx'], df['J1_fy'], df['J1_fz'])])
    j2 = np.array([[j2x, j2y, j2z] for j2x, j2y, j2z in zip(df['J2_fx'], df['J2_fy'], df['J2_fz'])])
    j3 = np.array([[j3x, j3y, j3z] for j3x, j3y, j3z in zip(df['J3_fx'], df['J3_fy'], df['J3_fz'])])    

    # Total joint forces
    forces = j1 + j2 + j3

    # Extract A matrices and compute net force vectors
    A = df['A'].values
    F = forces / mass - acl - g

    # Filter out invalid entries in A and F
    # Ensure A is a list of 3x3 matrices and F is a list of 3D vectors
    valid_indices = [i for i, a in enumerate(A) if isinstance(a, np.ndarray) and a.shape == (3, 3)]
    A_valid = [A[i] for i in valid_indices]
    F_valid = [F[i] for i in valid_indices]

    # initialize lists for filtered A matrices and F vectors
    A_filtered = []
    F_filtered = []

    # Compute the condition number and filter matrices
    for a, f in zip(A_valid, F_valid):
        if np.any(np.isnan(a)) or np.any(np.isnan(f)):
            continue
        cond = np.linalg.cond(a)
        if cond < 1e12:  # Filter based on condition number
            A_filtered.append(a)
            F_filtered.append(f)

    if not A_filtered:
        raise ValueError("No valid A matrices after filtering.")

    # Stack matrices and vectors for least-squares solution
    A_stacked = np.vstack(A_filtered)
    F_stacked = np.hstack(F_filtered)

    # Solve for the vector of coordinates (x, y, z) in the least-squares sense
    r_GP, residuals, rank, s = np.linalg.lstsq(A_stacked, F_stacked, rcond=None)
    r_PG = -r_GP # Invert the vector to get from P to G

    # Print the result in the console
    #print("r_GP = ", r_GP)

    return r_PG



In [17]:
def build_matrix_N(omega, alpha):
    """
    Compute the N matrix for each row in the dataframe.
    N is defined as:
    N = skew(omega) @ (I @ omega) + I @ alpha
    where I is the inertia tensor.
    Parameters
    ----------
    omega: array of angular velocities in the local reference frame (3,)
    alpha: array of angular accelerations in the local reference frame (3,)
    Returns
    ----------
    list of (3, 9) numpy arrays representing the torque mapping matrix
    for an asymmetric inertia tensor. Invalid entries are filled with NaN.
    """
    # Initialize the result list with NaN placeholders
    mat_n = [np.nan] * len(omega)

    # Loop through the omega and alpha arrays
    for i, (w, a) in enumerate(zip(omega, alpha)):
        # Skip rows with invalid omega or alpha
        if not isinstance(w, np.ndarray) or not isinstance(a, np.ndarray) or np.isnan(w).any() or np.isnan(a).any():
            continue

        wx, wy, wz = w # Unpack angular velocity components
        ax, ay, az = a # Unpack angular acceleration components

        omega_vec = np.array([wx, wy, wz]) # Angular velocity vector
        omega_skew = tilde(omega_vec) # Compute the skew symmetric matrix of omega

        # Linear part from I * alpha
        # Assuming inertia tensor I is diagonal with components ax, ay, az
        n_alpha = np.zeros((3, 9)) # Initialize the matrix for alpha contributions
        # Fill the diagonal blocks with the components of alpha
        n_alpha[0, 0:3] = [ax, ay, az] 
        n_alpha[1, 3:6] = [ax, ay, az]
        n_alpha[2, 6:9] = [ax, ay, az]

        # Matrix that maps J (flattened inertia) to I * omega
        I_omega = np.array([
            [wx, wy, wz,  0,  0,  0,  0,  0,  0],
            [ 0,  0,  0, wx, wy, wz,  0,  0,  0],
            [ 0,  0,  0,  0,  0,  0, wx, wy, wz]
        ])

        # Final N = skew(omega) @ (I @ omega) + I @ alpha
        n = omega_skew @ I_omega + n_alpha
        
        # append the computed matrix to the result list
        mat_n[i] = n

    return mat_n



In [18]:
def compute_torques(df, r1, r2, r3):
    """Compute torques based on the given dataframe and positions.
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the joint forces in the local reference frame.
    r1, r2, r3 : np.ndarray
        3D vectors representing the positions of the joints in the local reference frame.
    """
    # Extract the necessary columns from the DataFrame and arrange them into 3D vectors
    j1 = np.array([[j1x, j1y, j1z] for j1x, j1y, j1z in zip(df['J1_fx'], df['J1_fy'], df['J1_fz'])])
    j2 = np.array([[j2x, j2y, j2z] for j2x, j2y, j2z in zip(df['J2_fx'], df['J2_fy'], df['J2_fz'])])
    j3 = np.array([[j3x, j3y, j3z] for j3x, j3y, j3z in zip(df['J3_fx'], df['J3_fy'], df['J3_fz'])])    

    # Compute the torques using the cross product
    torque = np.cross(r1, j1) + np.cross(r2, j2) + np.cross(r3, j3)
    return torque



In [19]:
def compute_inertia(df):
    """
    Compute the inertia tensor components from the given DataFrame.
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the joint torques and angular velocities.
    Returns
    ----------
    inertia_components : np.ndarray
        Array of inertia tensor components in the order [Ixx, Ixy, Ixz, Iyx, Iyy, Iyz, Izx, Izy, Izz].
    inertia_tensor : np.ndarray
        3x3 inertia tensor matrix.
    Raises
    ----------  
    ValueError: If there are no valid pairs of N and T or if the dimensions of N and T are incompatible.
    ValueError: If the dimensions of N_stacked and T_stacked are incompatible.
    """
    # Extract the necessary columns from the DataFrame
    n = df['n'] # N matrix
    torques = np.array(df['torques'].tolist()) # Torques as a list of arrays
    
    # Initialize lists to store N matrices and torque vectors
    N = []
    Torques = []

    for i in range(len(df)):
        N.append(n[i])
        Torques.append(torques[i].flatten())  # Flatten the torque array to ensure compatibility

    # Filter out invalid entries in N and T simultaneously
    valid_pairs = [(n_elem, t_elem) for n_elem, t_elem in zip(N, Torques) if n_elem is not np.nan and t_elem is not np.nan]

    # Separate the filtered N and T
    if valid_pairs:  # Ensure there are valid pairs
        N_filtered, T_filtered = zip(*valid_pairs)
        N_filtered = np.array(N_filtered)  # Convert to NumPy array
        T_filtered = np.array(T_filtered)  # Convert to NumPy array
    else:
        raise ValueError('No valid pairs found for N and T.')

    # Reshape N_filtered to match T_filtered
    N_stacked = N_filtered.reshape(-1, N_filtered.shape[-1])
    # Reshape T_filtered to match the number of rows in N_stacked
    T_stacked = T_filtered.flatten()

    # Solve for the inertia tensor components in the least-squares sense
    if N_stacked.shape[0] != T_stacked.shape[0]:
        raise ValueError(f'Incompatible dimensions: N_stacked has {N_stacked.shape[0]} rows, T_stacked has {T_stacked.shape[0]} rows.')

    inertia_components, residuals, rank, s = np.linalg.lstsq(N_stacked, T_stacked, rcond=None)

    # Rearrange the components of the inertia into a 3x3 inertia tensor.
    # the least-squares solution gives us the components in a (9,) vector in the order
    # [Ixx, Ixy, Ixz, Iyx, Iyy, Iyz, Izx, Izy, Izz]

    inertia_tensor = np.array(
        [inertia_components[0], inertia_components[1], inertia_components[2],
         inertia_components[3], inertia_components[4], inertia_components[5],
         inertia_components[6], inertia_components[7], inertia_components[8]
         ]).reshape(3, 3) # ensure the inertia tensor is a 3x3 matrix
    
    return inertia_components, inertia_tensor



In [20]:
def bias_force_reading(force_list, bias):
    """
    Modify the force readings by adding or substracting a percentual bias.
    """
    # Convert the bias percentage to a decimal
    bias_decimal = bias / 100.0
    
    # Apply the bias to each force reading
    biased_forces = [force * (1 + bias_decimal) for force in force_list]
    
    return biased_forces


In [21]:
def static_cog(r1, r2, r3, F1, F2, F3, mass):
    """
    Compute the center of gravity (CoG) based on the static equilibrium of forces and torques.
    Parameters
    ----------
    r1, r2, r3 : np.ndarray
        3D vectors representing the positions of the spherical joints in the local reference frame.
    F1, F2, F3 : np.ndarray
        3D vectors representing the forces at the spherical joints in the local reference frame.
    mass: float
        Mass of the body [kg].
    Returns
    -------
    np.ndarray
        The computed center of gravity (CoG) in the local reference frame.
    """
    
    # Static equilibrium aroung the CoG
    # Compute the sum of forces at the joints
    sum_forces = -(F1 + F2 + F3)  # Sum of forces at the joints
    # Compute the sum of torques at the joints
    sum_torques = np.cross(r1, F1) + np.cross(r2, F2) + np.cross(r3, F3)  # Sum of torques at the joints
    # Compute the skew-symmetric matrix of the sum of forces
    skew_forces = tilde(sum_forces)  # Skew-symmetric matrix of the sum of forces
    # Solve for the vector from the reference point P to the center of gravity G
    r_G, residuals, rank, s = np.linalg.lstsq(skew_forces, sum_torques, rcond=None)  # Solve for the vector from the reference point P to the center of gravity G

    return r_G


In [22]:
def interpolate_stair_like(y):
    """
    Interpolate a stair-like signal to create a smooth curve at the same sampling rate.
    Parameters
    ----------
    y : array-like
        The stair-like signal to be interpolated.
    Returns
    -------     
    y_interp : np.ndarray
        Iinterpolated signal."""
    
    # 
    x = np.arange(len(y))
    # Detect where the value changes
    change_indices = np.where(np.diff(y) != 0)[0] + 1

    # Identify boundaries of flat regions
    flat_regions = np.split(x, change_indices)
    flat_values = np.split(y, change_indices)

    # Step 2: Compute midpoints of each flat region
    mid_x = np.array([region[len(region) // 2] for region in flat_regions])
    mid_y = np.array([values[0] for values in flat_values])  # all values in a region are equal

    # Step 3: Interpolate from midpoints
    y_linear_mid = np.interp(x, mid_x, mid_y)

    return y_linear_mid


In [74]:
def encoder(angle_reading, PPR, interpolate = True):
    """
    Simulate an encoder reading based on the angle reading.
    Parameters
    ----------
    angle_reading : np.ndarray
        The angle reading in radians.
    PPR : int
        Pulses per revolution of the encoder.
    Returns
    -------
    np.ndarray
        The decoded angle readings from the encoder.
    """
    sensitivity = 2 * np.pi / PPR

    encoder_reading = np.zeros_like(angle_reading)

    for i in range(len(angle_reading)):
        if angle_reading[i] < 0:
            encoder_reading[i] = np.ceil(angle_reading[i] / sensitivity) * sensitivity     
        else:
            encoder_reading[i] = np.floor(angle_reading[i] / sensitivity) * sensitivity

    if interpolate:
        # interpolate the encoder reading
        encoder_reading = interpolate_stair_like(encoder_reading)

    
    return encoder_reading


In [24]:
def main(data_dynamic, data_static, mass, r_s1, r_s2, r_s3, r_u1, r_u2, r_u3, v_1, v_2, v_3, 
         gravity = np.array([0., -9.80665, 0.]), actual_cog = False, 
         bias_force1 = 0, bias_force2 = 0, bias_force3 = 0, PPR = 0, angle_plots = False,
         print_results = True):
        
        """Main function to process the data and compute the required values.
        Parameters:
        ---------- 
        data (str): Path to the data file.
        mass (float): Mass of the system [kg].
        r_s1 (array): Position of spherical joint 1 in the local frame [m].
        r_s2 (array): Position of spherical joint 2 in the local frame [m].
        r_s3 (array): Position of spherical joint 3 in the local frame [m].
        r_u1 (array): Position of U joint 1 in the global frame [m].
        r_u2 (array): Position of U joint 2 in the global frame [m].
        r_u3 (array): Position of U joint 3 in the global frame [m].
        v_1 (array): Vector that defines the original orientation of rod 1.
        v_2 (array): Vector that defines the original orientation of rod 2.
        v_3 (array): Vector that defines the original orientation of rod 3.
        gravity (array): Gravity vector in the global frame. Default is [0., -9.80665, 0.], unit = m/s^2.
        available options are:
        actual_cog (bool): If True, compute the Inertia tensor using the real coordinates of the CoG
        the actual CoG is hardcoded to be at the origin of the global reference frame (0, 0, 0).
        bias_force1 (float): Percentage of the force at joint 1 to be added as a bias.
        bias_force2 (float): Percentage of the force at joint 2 to be added as a bias.
        bias_force3 (float): Percentage of the force at joint 3 to be added as a bias.
        PPR (int): Pulses per revolution of the simulated encoder.
        """

        # Read the data from thes file and create the 2 DataFrame
        df_dynamic = pd.read_csv(data_dynamic, sep='\s+', skiprows=1, header=0)
        df_dynamic.columns = df_dynamic.columns.str.strip()  # remove leading/trailing whitespace for column names

        df_static = pd.read_csv(data_static, sep='\s+', skiprows=1, header=0)
        df_static.columns = df_static.columns.str.strip()  # remove leading/trailing whitespace for column names

        # Define the required columns for the DataFrame
        # These columns are expected to be present in the DataFrame
        # U1Rx, U1Rz, U2Rx, U2Rz, U3Rx, U3Rz are the angles of the U-joints around the x and z axes
        # J1_fx, J1_fy, J1_fz, J2_fx, J2_fy, J2_fz, J3_fx, J3_fy, J3_fz are the forces at the spherical joints
        required_columns_dynamic = ['U1Rx', 'U1Rz', 'U2Rx', 'U2Rz', 'U3Rx', 'U3Rz', 'J1_fx', 'J1_fy', 'J1_fz',
                            'J2_fx', 'J2_fy', 'J2_fz', 'J3_fx', 'J3_fy', 'J3_fz',]
        
        # Check if all required columns are present, 
        # if any are missing, raise an error and print the missing and current columns
        if not all(col in df_dynamic.columns for col in required_columns_dynamic):
                missing_columns = [col for col in required_columns_dynamic if col not in df_dynamic.columns]
                raise ValueError(f"Dynamic DataFrame is missing the columns: {missing_columns}. "
                        f"Currently present columns: {list(df_dynamic.columns)}. ")
        
        # Define the required columns for the static DataFrame
        # These columns are expected to be present in the DataFrame, force at the spherical joints in the local reference frame
        required_columns_static = ['J1_fx', 'J1_fy', 'J1_fz', 'J2_fx', 'J2_fy', 'J2_fz', 'J3_fx', 'J3_fy', 'J3_fz',]

        # Check if all required columns are present,
        # if any are missing, raise an error and print the missing and current columns
        if not all(col in df_static.columns for col in required_columns_static):
                missing_columns = [col for col in required_columns_static if col not in df_static.columns]
                raise ValueError(f"Static DataFrame is missing the columns: {missing_columns}. "
                        f"Currently present columns: {list(df_static.columns)}. ")

        if bias_force1 != 0:
        # Apply the bias to the forces at the spherical joints
                for component in ['fx', 'fy', 'fz']:
                        df_dynamic[f'J1_{component}'] = bias_force_reading(df_dynamic[f'J1_{component}'], bias_force1)
                        df_static[f'J1_{component}'] = bias_force_reading(df_static[f'J1_{component}'], bias_force1)    

        if bias_force2 != 0:
        # Apply the bias to the forces at the spherical joints
                for component in ['fx', 'fy', 'fz']:
                        df_dynamic[f'J2_{component}'] = bias_force_reading(df_dynamic[f'J2_{component}'], bias_force2)
                        df_static[f'J2_{component}'] = bias_force_reading(df_static[f'J2_{component}'], bias_force2)
        if bias_force3 != 0:
        # Apply the bias to the forces at the spherical joints
                for component in ['fx', 'fy', 'fz']:
                        df_dynamic[f'J3_{component}'] = bias_force_reading(df_dynamic[f'J3_{component}'], bias_force3)
                        df_static[f'J3_{component}'] = bias_force_reading(df_static[f'J3_{component}'], bias_force3)

        # Extract the joint forces from the static DataFrame
        j1_static = df_static.loc[0, ['J1_fx', 'J1_fy', 'J1_fz']].values
        j2_static = df_static.loc[0, ['J2_fx', 'J2_fy', 'J2_fz']].values
        j3_static = df_static.loc[0, ['J3_fx', 'J3_fy', 'J3_fz']].values

        r_G = static_cog(r_s1, r_s2, r_s3, j1_static, j2_static, j3_static, mass)     

        time = df_dynamic['Time'].values  # Extract time from the DataFrame

        # Extract the angles for the U-joints from the DataFrame
        if PPR > 0:
                # Transform the angle readings from high resolution simulation to encoder counts 
                # and decode them back to angles
                PPR = int(PPR)  # Ensure PPR is an integer
                U1_angle_x = encoder(df_dynamic['U1Rx'].values, PPR)
                U1_angle_z = encoder(df_dynamic['U1Rz'].values, PPR)
                U2_angle_x = encoder(df_dynamic['U2Rx'].values, PPR)
                U2_angle_z = encoder(df_dynamic['U2Rz'].values, PPR)
                U3_angle_x = encoder(df_dynamic['U3Rx'].values, PPR)
                U3_angle_z = encoder(df_dynamic['U3Rz'].values, PPR)
        else:
                # If PPR is not set, use the original angles from the DataFrame
                U1_angle_x = df_dynamic['U1Rx'].values
                U1_angle_z = df_dynamic['U1Rz'].values
                U2_angle_x = df_dynamic['U2Rx'].values
                U2_angle_z = df_dynamic['U2Rz'].values
                U3_angle_x = df_dynamic['U3Rx'].values
                U3_angle_z = df_dynamic['U3Rz'].values

        # Compute the rotation matrices for each rod
        U1_rotation_matrices = compute_rotations(U1_angle_x, U1_angle_z)
        U2_rotation_matrices = compute_rotations(U2_angle_x, U2_angle_z)
        U3_rotation_matrices = compute_rotations(U3_angle_x, U3_angle_z)

        # Compute the positions of the spherical joints after each rotation
        P1 = compute_positions(U1_rotation_matrices, v_1, r_u1)
        P2 = compute_positions(U2_rotation_matrices, v_2, r_u2)
        P3 = compute_positions(U3_rotation_matrices, v_3, r_u3)

        # Compute the rotation matrices for the body
        body_rotation_matrices = compute_body_orientation(P1, P2, P3)

        # Compute the angular velocities and accelerations of the body in the global frame
        body_angular_velocities = compute_body_omega(body_rotation_matrices, df_dynamic)
        body_angular_accelerations = compute_body_alpha(body_angular_velocities, df_dynamic)

        # Compute the angular velocities and accelerations in the local frame
        local_angular_velocities = transform_global_to_local(body_angular_velocities, body_rotation_matrices)
        local_angular_accelerations = transform_global_to_local(body_angular_accelerations, body_rotation_matrices)

        # Build the N matrix for computing the Interia tensor
        df_dynamic['n'] = build_matrix_N(local_angular_velocities, local_angular_accelerations)

        # Define which coordinates of CoG to use for the inertia tensor
        if actual_cog:
                # Use the actual CoG at the origin of the global reference frame
                r1 = np.array([-0.45, 0.5, -0.25]) # vector from CoG to joint 1
                r2 = np.array([0.45, 0.5, -0.25]) # vector from CoG to joint 2
                r3 = np.array([-0.25, 0.5, 0.5]) # vector from CoG to joint 3
        else:
                # Use the computed CoG from the data
                inertia_cog = r_G 
                r1 = r_s1 - r_G # vector from CoG to joint 1
                r2 = r_s2 - r_G # vector from CoG to joint 2
                r3 = r_s3 - r_G # vector from CoG to joint 3
        
        # Compute torques
        torques = compute_torques(df_dynamic, r1, r2, r3)

        # Add torques to the DataFrame
        df_dynamic['torques'] = torques.tolist()  # Save as a list of vectors

        # Compute the inertia tensor components and the inertia tensor itself
        inertia_components, inertia_tensor = compute_inertia(df_dynamic)

        # Add the computed values to the DataFrame
        # Joint positions
        # Joint 1
        df_dynamic['P1_x'] = P1[:, 0]
        df_dynamic['P1_y'] = P1[:, 1]
        df_dynamic['P1_z'] = P1[:, 2]
        # Joint 2
        df_dynamic['P2_x'] = P2[:, 0]
        df_dynamic['P2_y'] = P2[:, 1]
        df_dynamic['P2_z'] = P2[:, 2]
        # Joint 3
        df_dynamic['P3_x'] = P3[:, 0]
        df_dynamic['P3_y'] = P3[:, 1]
        df_dynamic['P3_z'] = P3[:, 2]
        # angular velocities in the global frame
        df_dynamic['omega_x_global'] = body_angular_velocities[:, 0]
        df_dynamic['omega_y_global'] = body_angular_velocities[:, 1]
        df_dynamic['omega_z_global'] = body_angular_velocities[:, 2]
        # angular accelerations in the global frame
        df_dynamic['alpha_x_global'] = body_angular_accelerations[:, 0]
        df_dynamic['alpha_y_global'] = body_angular_accelerations[:, 1]
        df_dynamic['alpha_z_global'] = body_angular_accelerations[:, 2]
        # angular velocities in the local frame
        df_dynamic['omega_x_local'] = local_angular_velocities[:, 0]
        df_dynamic['omega_y_local'] = local_angular_velocities[:, 1]
        df_dynamic['omega_z_local'] = local_angular_velocities[:, 2]
        # angular accelerations in the local frame
        df_dynamic['alpha_x_local'] = local_angular_accelerations[:, 0]
        df_dynamic['alpha_y_local'] = local_angular_accelerations[:, 1]
        df_dynamic['alpha_z_local'] = local_angular_accelerations[:, 2]
        # angles of the U-joints from the encoder
        df_dynamic['U1_Rx_encoder'] = U1_angle_x
        df_dynamic['U1_Rz_encoder'] = U1_angle_z
        df_dynamic['U2_Rx_encoder'] = U2_angle_x
        df_dynamic['U2_Rz_encoder'] = U2_angle_z
        df_dynamic['U3_Rx_encoder'] = U3_angle_x
        df_dynamic['U3_Rz_encoder'] = U3_angle_z

        # rotation matrices for the body
        df_dynamic['body_rotation_matrix'] = list(body_rotation_matrices)
        
        if print_results:
                # Print the computed values in the console
                print(f"Computed center of gravity (CoG) vector r_G in the local referece frame:\n{r_G}")
                print("")
                print(f"Inertia tensor:\n{inertia_tensor}")

        if angle_plots:
                # Plot the angles of the U-joints in a 3x2 grid (columns: Rx/Rz, rows: U1/U2/U3)
                fig, axes = plt.subplots(3, 2, figsize=(12, 10), sharex=True)
                joint_labels = ['U1', 'U2', 'U3']
                angle_x = [U1_angle_x, U2_angle_x, U3_angle_x]
                angle_z = [U1_angle_z, U2_angle_z, U3_angle_z]
                sim_x = [df_dynamic['U1Rx'], df_dynamic['U2Rx'], df_dynamic['U3Rx']]
                sim_z = [df_dynamic['U1Rz'], df_dynamic['U2Rz'], df_dynamic['U3Rz']]

                for i in range(3):
                        # Rx (column 0)
                        axes[i, 0].plot(time, sim_x[i], label=f'{joint_labels[i]} Rx simulation')
                        axes[i, 0].plot(time, angle_x[i], '--', label=f'{joint_labels[i]} Rx decoded')
                        axes[i, 0].set_title(f'{joint_labels[i]} Rx')
                        axes[i, 0].set_ylabel('Angle [rad]')
                        axes[i, 0].legend()
                        axes[i, 0].grid(True)
                        # Rz (column 1)
                        axes[i, 1].plot(time, sim_z[i], label=f'{joint_labels[i]} Rz simulation')
                        axes[i, 1].plot(time, angle_z[i], '--', label=f'{joint_labels[i]} Rz decoded')
                        axes[i, 1].set_title(f'{joint_labels[i]} Rz')
                        axes[i, 1].set_xlabel('Time [s]')
                        axes[i, 1].set_ylabel('Angle [rad]')
                        axes[i, 1].legend()
                        axes[i, 1].grid(True)

                plt.suptitle('Angles of the U-joints over time')
                plt.tight_layout(rect=[0, 0, 1, 0.96])
                plt.show()

        # return the DataFrame, center of gravity, and inertia tensor
        return df_dynamic, r_G, inertia_tensor


# SECTION: Helper functions for debugging and visualization


In [25]:
def plot_local_and_global_frames(R, vecs, labels, title="Frame Comparison", scale=1.0):
    """
    Plot local body frame, global frame (colored), and vectors in local frame.

    Parameters:
    - R: 3x3 body rotation matrix at a given timestep
    - vecs: list of vectors in local frame
    - labels: corresponding labels for each vector
    - title: plot title
    - scale: arrow length scaling factor
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    origin = np.zeros(3)

    # Plot global frame axes (distinct colors)
    global_colors = ['violet', 'lime', 'aqua']
    for i, color in zip(range(3), global_colors):
        ax.quiver(*origin, *(np.eye(3)[:, i] * scale), color=color, linestyle='-', label=f'Global {["X", "Y", "Z"][i]}')

    # Plot local (body) frame axes (RGB)
    local_colors = ['r', 'g', 'b']
    for i, color in zip(range(3), local_colors):
        ax.quiver(*origin, *(R[:, i] * scale), color=color, linestyle='-', label=f'Local {["X", "Y", "Z"][i]}')

    # Plot any additional vectors (e.g. gravity)
    for vec, label in zip(vecs, labels):
        ax.quiver(*origin, *(vec * scale), color='k', linestyle='dashed', label=label)

    ax.set_xlim([-scale, scale])
    ax.set_ylim([-scale, scale])
    ax.set_zlim([-scale, scale])
    ax.set_title(title)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.legend()
    plt.tight_layout()
    plt.show()



In [26]:
def plot_magnitude_error(time, computed, simulated, title="Body Magnitude", units="[]", tolerance=1e-6):
    """
    Plots computed vs. simulated components, absolute error, and percentage error of any given variable.

    Parameters:
    - time: array of time values
    - computed: Nx3 array of computed velocity components
    - simulated: Nx3 array of simulated velocity components
    - title: Title for the plot 
    - units: units of the variable
    - tolerance: threshold to avoid division by zero in percent error
    ----------
    This function creates a 3x3 grid of subplots:
    - First column: Computed vs. Simulated components
    - Second column: Absolute error
    - Third column: Percent error
    ----------
    Sample usage, plotting the position of Joint 1:
    plot_magnitude_error(times, P1, P1_sim, title="Joint 1 Position", units="[m]")
    """
    component_labels = ['x', 'y', 'z']
    fig, axes = plt.subplots(3, 3, figsize=(15, 5), sharex=True)
    fig.suptitle(f"{title} {units}", fontsize=14)
    points_removed_any = False
    
    for i in range(3):
        # Absolute error
        error = computed[:, i] - simulated[:, i]

        # Percent error with tolerance check
        mask = (np.abs(simulated[:, i]) > tolerance)
        percent_error = np.where(
            mask,
            (error / simulated[:, i]) * 100,
            0.0
        )
        points_removed = not np.all(mask)
        if points_removed:
            points_removed_any = True
        

        # Plot computed vs. simulated
        axes[i, 0].plot(time, simulated[:, i], 'r', label='simulation')
        axes[i, 0].plot(time, computed[:, i], 'k--', label='computed')
        axes[i, 0].set_ylabel(f"{component_labels[i]}-component")
        axes[i, 0].legend()

        # Absolute error
        axes[i, 1].plot(time, error, color='dodgerblue')
        axes[i, 1].set_ylabel(f"Error {units}")

        # Percent error
        axes[i, 2].plot(time, percent_error, color='steelblue')
        axes[i, 2].set_ylabel("Error %")

    axes[2, 0].set_xlabel("Time [s]")
    axes[2, 1].set_xlabel("Time [s]")
    axes[2, 2].set_xlabel("Time [s]")

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
    
    if points_removed_any:
        print(f"Points where simulated values were near zero (tolerance = {tolerance}) have been removed from the percent error plots.") 


In [27]:
def plot_angles(angle_true, angle_decoded, time):

    # Plot the true and decoded angles
    plt.plot(time, angle_true, label="True Angle", linewidth=2)
    plt.plot(time, angle_decoded, '--', label="Decoded from A/B")
    plt.xlabel("Time [s]")
    plt.ylabel("Angle [rad]")
    plt.title("True vs Decoded Angle (Aligned by Time)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_encoder_interp(time, true_angle, encoded, interpolated, PPR):
    """
    Plot the encoder interpolation results.
    """
    plt.figure(figsize=(10, 5))
    plt.plot(time, true_angle, label='True Angle', color='blue')
    plt.plot(time, encoded, label='Encoded Angle', linestyle='--', color='orange')
    plt.plot(time, interpolated, label='Interpolated Angle', linestyle=':', color='green')
    plt.xlabel('Time [s]')
    plt.ylabel('Angle [rad]')
    plt.title('Angle of the U-joint with Encoder Interpolation')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# SECTION: Main execution block to run the script



In [28]:
mass = 7801.0  # Mass of the body in kg

# Define the positions of the U-joints in the global frame
r_u_joint_1 = np.array([-1.05, 0.8, -0.4])
r_u_joint_2 = np.array([1.05, 0.8, -0.4])
r_u_joint_3 = np.array([-0.85, 0.8, 0.65])

# Define the rod vectors
r_vector1 = np.array([0.0, -0.687, 0.0])
r_vector2 = np.array([0.0, -0.687, 0.0])
r_vector3 = np.array([0.0, -0.687, 0.0])

# Define the positions of the spherical joints in the local frame
r_spherical_joint1 = np.array([-0.45, 0.5, -0.25])  # Joint 1 position in local frame
r_spherical_joint2 = np.array([0.45, 0.5, -0.25])   # Joint 2 position in local frame
r_spherical_joint3 = np.array([-0.25, 0.5, 0.5])   # Joint 3 position in local frame


Data directly from the simulation, using the known true location of the CoG for computing the inertia Tensor:

In [29]:
filepath_dynamic = "data/dynamic_100.tab"  # Path to the dynamic data file
filepath_static = "data/static_100.tab"  # Path to the static data file

df, cog, J = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3)

print(f"Inertia tensor:\n{J}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[-3.37995522e-08 -4.26591055e-09 -5.21169647e-08]

Inertia tensor:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]
Inertia tensor:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


Using the statically computed CoG

In [30]:
df_comp_cg, cog_comp_cg, J_comp_cg = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3, 
                       actual_cog=False)

print(f"Inertia tensor:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[-3.37995522e-08 -4.26591055e-09 -5.21169647e-08]

Inertia tensor:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]
Inertia tensor:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


Biasing the reading of one of the load cells by +1%


In [31]:
df1, cog1, J1 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3, 
                       actual_cog=False, bias_force1 = 1, bias_force2 = 0, bias_force3 = 0, PPR = 0)

print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[ 1.52184506e-03 -7.13634248e-05 -9.69953009e-04]

Inertia tensor:
[[1.31066621e+03 3.68216821e+00 4.57352257e-02]
 [5.94439237e+00 1.32176975e+03 3.29802862e-01]
 [2.59576794e+00 3.44956179e+00 1.31152135e+03]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


Biasing the load cells readings by +1%, -1% and 0% 

In [32]:
df2, cog2, J2 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3, 
                       actual_cog=False, bias_force1 = 1, bias_force2 = -1, bias_force3 = 0, PPR = 0)

print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[ 3.85372651e-03 -7.82719382e-05 -1.66874554e-03]

Inertia tensor:
[[ 1.30527259e+03  4.26402158e+00 -1.38279887e+00]
 [ 1.35363734e+01  1.32253115e+03 -1.73099262e-01]
 [-1.90928916e-01  3.49677574e+00  1.30408135e+03]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


Biasing the load cells readings by +1%, -1% and +1% 

In [33]:
df3, cog3, J3 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3, 
                       actual_cog=False, bias_force1 = 1, bias_force2 = -1, bias_force3 = +1, PPR = 0)

print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[ 4.64505837e-03 -2.57115409e-05 -1.36144478e-03]

Inertia tensor:
[[ 1.30949917e+03  1.27954117e+00 -2.50753914e+00]
 [ 1.59247768e+01  1.32749728e+03 -3.14322389e+00]
 [ 4.49464333e-01 -1.72690779e+00  1.30618409e+03]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


Biasing the load cells readings by +1%, -1% and -1% 


In [34]:
df4, cog4, J4 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3, 
                       actual_cog=False, bias_force1 = 1, bias_force2 = -1, bias_force3 = -1, PPR = 0)

print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[ 0.00306335 -0.00012702 -0.00197099]

Inertia tensor:
[[ 1.30103631e+03  7.24976152e+00 -2.51687585e-01]
 [ 1.11464639e+01  1.31755193e+03  2.79231441e+00]
 [-8.24397179e-01  8.71708619e+00  1.30197746e+03]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


Biasing the load cells readings by -1%, -1% and -1% 

In [35]:
df5, cog5, J5 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3, 
                       actual_cog=False, bias_force1 = -1, bias_force2 = -1, bias_force3 = -1, PPR = 0)

print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[-3.37995523e-08 -4.26591054e-09 -5.21169647e-08]

Inertia tensor:
[[ 1.29412356e+03  1.12672836e-01  3.42259644e-01]
 [ 7.40162115e-01  1.29974350e+03 -2.05912467e+00]
 [ 5.91609018e+00 -1.75243617e+00  1.29493950e+03]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


Biasing the load cells readings by +1%, +1% and +1% 


In [36]:
df6, cog6, J6 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3, 
                       actual_cog=False, bias_force1 = 1, bias_force2 = 1, bias_force3 = 1, PPR = 0)

print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[-3.37995521e-08 -4.26591055e-09 -5.21169647e-08]

Inertia tensor:
[[ 1.32026747e+03  1.14949055e-01  3.49173980e-01]
 [ 7.55114885e-01  1.32600095e+03 -2.10072315e+00]
 [ 6.03560716e+00 -1.78783892e+00  1.32109990e+03]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


Biasing the load cells readings by -1.5%, 0% and +1% 

In [37]:
df7, cog7, J7 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3, 
                       actual_cog=False, bias_force1 = 1.5, bias_force2 = 0, bias_force3 = 1, PPR = 0)

print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[ 3.06588099e-03 -5.82652849e-05 -1.16110049e-03]

Inertia tensor:
[[ 1.31663435e+03  2.48712003e+00 -1.23431206e+00]
 [ 1.09397270e+01  1.33120087e+03 -1.41733515e+00]
 [ 1.54065049e+00  8.48821361e-01  1.31538224e+03]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


Simulating encoder readings

In [78]:
df9, cog9, J9 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3,
                       bias_force1 = 0, bias_force2 = 0, bias_force3 = 0, PPR = 2700, angle_plots=False, 
                       print_results=True)

print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[-3.37995522e-08 -4.26591055e-09 -5.21169647e-08]

Inertia tensor:
[[ 970.25107765   -8.83401306  104.6140861 ]
 [  -5.42498259 1072.27685855  -13.68354721]
 [  97.03182318  -14.5953472  1089.56052532]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


In [39]:
df10, cog10, J10 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3,
                       bias_force1 = 0, bias_force2 = 0, bias_force3 = 0, PPR = 3600, angle_plots=False, 
                       print_results=True)
print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[-3.37995522e-08 -4.26591055e-09 -5.21169647e-08]

Inertia tensor:
[[1113.45058843  -12.79208786   48.03111275]
 [  -7.54447102 1151.16297115    3.43099082]
 [  52.69659266    7.24987414 1187.974546  ]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


In [40]:
df11, cog11, J11 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3,
                       bias_force1 = 0, bias_force2 = 0, bias_force3 = 0, PPR = 7200, angle_plots=False, 
                       print_results=True)
print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[-3.37995522e-08 -4.26591055e-09 -5.21169647e-08]

Inertia tensor:
[[ 1.24914622e+03  4.19410766e-02  1.49530958e+01]
 [ 5.14002997e+00  1.26757247e+03 -1.48184232e+00]
 [ 2.02106958e+01  3.23167323e-01  1.27054880e+03]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


In [85]:
df12, cog12, J12 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3,
                       bias_force1 = 0, bias_force2 = 0, bias_force3 = 0, PPR = 14400, angle_plots=False, 
                       print_results=True)
print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[-3.37995522e-08 -4.26591055e-09 -5.21169647e-08]

Inertia tensor:
[[ 1.28715306e+03 -9.18240896e-01  7.08195500e+00]
 [ 8.81586436e-01  1.29663672e+03 -2.19414637e+00]
 [ 1.15402885e+01 -2.94861419e+00  1.29362854e+03]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


Combined errors: +1%, -1%, +1% bias and PPR 3600

In [83]:
df11, cog11, J11 = main(filepath_dynamic, filepath_static, mass, r_spherical_joint1, r_spherical_joint2, r_spherical_joint3, 
                       r_u_joint_1, r_u_joint_2, r_u_joint_3, r_vector1, r_vector2, r_vector3,
                       bias_force1 = 1, bias_force2 = -1, bias_force3 = +1, PPR = 14400, angle_plots=False, 
                       print_results=True)
print("")
print(f"Inertia tensor with perfect readings:\n{J_comp_cg}")


Computed center of gravity (CoG) vector r_G in the local referece frame:
[ 4.64505837e-03 -2.57115409e-05 -1.36144478e-03]

Inertia tensor:
[[ 1.28939415e+03  2.72732753e-01  4.30964237e+00]
 [ 1.58623741e+01  1.31105084e+03 -3.20671067e+00]
 [ 6.08988337e+00 -2.94789665e+00  1.29182589e+03]]

Inertia tensor with perfect readings:
[[ 1.30719551e+03  1.13810945e-01  3.45716812e-01]
 [ 7.47638500e-01  1.31287222e+03 -2.07992391e+00]
 [ 5.97584867e+00 -1.77013754e+00  1.30801970e+03]]


In [84]:
J11-J_comp_cg


array([[-17.80136244,   0.15892181,   3.96392556],
       [ 15.11473564,  -1.82138728,  -1.12678677],
       [  0.1140347 ,  -1.17775911, -16.19380936]])

In [82]:
delta_deg = 0.025  # Encoder resolution in degrees
PPR_deg = int(360 / delta_deg)
PPR_deg


14400

In [44]:
mass


7801.0